In [42]:
# some useful mysklearn package import statements and reloads
import importlib

import pandas as pd
import numpy as np
import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

# uncomment once you paste your myclassifiers.py into mysklearn package
import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, RandomForest, MyDummyClassifier, MyNaiveBayesClassifier, MyDecisionTreeClassifier, MyRandomForestClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder


# ✈️ Flight Delay Classification ✈️

In [43]:
import pandas as pd
from sklearn.model_selection import KFold

# Load data
data = pd.read_csv('normalized_flights.csv')

# Compute flight delay (in minutes, assuming sched_dep_time and dep_time are timestamps or integers)
data['flight_delay'] = data['sched_dep_time'] - data['dep_time']

# Extract meaningful features for predicting delays
X = data[[
    'year', 'month', 'day', 'hour', 'minute',  # Time-based features
    'carrier', 'flight', 'tailnum',            # Operational features
    'origin', 'dest', 'distance', 'air_time', # Location & flight characteristics
    'dep_delay', 'sched_dep_time'             # Existing delay and timing
]]

# Target variable: flight delay
y = data['flight_delay']

# Initialize K-Fold cross-validation (10 splits)
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Placeholder for performance metrics
accuracies, error_rates, precisions, recalls, f1_scores, confusion_matrices = [], [], [], [], [], []


In [44]:
import numpy as np
import pandas as pd

# Load the dataset (assuming `data` is a pandas DataFrame containing the provided sample data)
data = pd.read_csv('normalized_flights.csv') 

# Define delay categories
def categorize_delay(delay):
    if delay <= 0:
        return 0
    elif 0 < delay <= 30:
        return 1
    elif 30 < delay <= 60:
        return 2
    elif 60 < delay <= 120:
        return 3
    elif 120 < delay <= 180:
        return 4
    else:
        return 5

# Map delays to categories
data['delay_category'] = data['arr_delay'].apply(categorize_delay)

# Convert categorical features to integers (label encoding)
categorical_cols = ['carrier', 'flight', 'tailnum', 'origin', 'dest']  # List your categorical columns
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str))  # Convert to string if necessary and apply label encoding
    label_encoders[col] = le  # Store the encoder to use it later if needed

# Prepare features (X) and labels (y)
X = data[['dep_time', 'sched_dep_time', 'dep_delay', 'arr_time', 'sched_arr_time', 'arr_delay', 'air_time', 'distance', 'dest',
          'carrier', 'month', 'hour']].fillna(0)

y = data['delay_category'].values

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


### kNN Performance Evaluation & Confusion Matrix

In [45]:
import numpy as np
import pandas as pd

X = np.array(X) 
y = np.array(y) 

# Initialize StratifiedKFold with k=10 (assuming myevaluation provides this utility)
kf = myevaluation.stratified_kfold_split(X, y, n_splits=10, random_state=None, shuffle=False)

# Initialize lists to store metrics (accuracy, precision, recall, f1, confusion matrices, etc.)
knn_accuracies, knn_precisions, knn_recalls, knn_f1s, knn_conf_matrices = [], [], [], [], []

# Initialize lists to gather true and predicted values across all folds
all_y_true = []
all_y_pred = []

# Define the correct labels for the confusion matrix and metrics
labels = [0, 1, 2, 3, 4, 5]

# Loop over each fold
for train_index, test_index in kf:
    knn_X_train, knn_X_test = X[train_index], X[test_index]
    knn_y_train, knn_y_test = y[train_index], y[test_index]

    # Initialize and fit MyKNeighborsClassifier
    knn_model = MyKNeighborsClassifier()
    knn_model.fit(knn_X_train.tolist(), knn_y_train.tolist())
    
    # Make predictions on the test set
    knn_y_pred = knn_model.predict(knn_X_test.tolist())
    
    # Calculate and store metrics for this fold
    accuracy = myevaluation.accuracy_score(knn_y_test, knn_y_pred)
    precision_score = myevaluation.binary_precision_score(knn_y_test, knn_y_pred, labels= labels)  # Multi-class precision
    recall_score = myevaluation.binary_recall_score(knn_y_test, knn_y_pred, labels= labels)  # Multi-class recall
    f1 = myevaluation.binary_f1_score(knn_y_test, knn_y_pred, labels= labels)  # Multi-class F1 score
    confusion_matrix = myevaluation.confusion_matrix(knn_y_test, knn_y_pred, labels=labels)

    # Store fold metrics
    knn_accuracies.append(accuracy)
    knn_precisions.append(precision_score)
    knn_recalls.append(recall_score)
    knn_f1s.append(f1)
    knn_conf_matrices.append(confusion_matrix)
    
    # Collect all true and predicted labels for final confusion matrix
    all_y_true.extend(knn_y_test)
    all_y_pred.extend(knn_y_pred)

# Calculate the final confusion matrix using all folds' predictions
final_confusion_matrix = myevaluation.confusion_matrix(all_y_true, all_y_pred, labels=labels)
matrix_with_totals = myutils.calculate_confusion_matrix_totals(final_confusion_matrix)

In [46]:
print("===========================================")
print("kNN Classifier Performance")
print("===========================================")

# Calculate and print average performance
print(f"Accuracy: {np.mean(knn_accuracies):.2f}")
print(f"Error: {1 - np.mean(knn_accuracies):.2f}")
print(f"Precision: {np.mean(knn_precisions):.2f}")
print(f"Recall: {np.mean(knn_recalls):.2f}")
print(f"F1 Score: {np.mean(knn_f1s):.2f}\n")

# Display the confusion matrix
print("Confusion Matrix:")
myutils.display_confusion_matrix(matrix_with_totals, labels)

kNN Classifier Performance
Accuracy: 0.45
Error: 0.55
Precision: 0.48
Recall: 0.74
F1 Score: 0.57

Confusion Matrix:
  Survived    0    1    2    3    4    5    Total    Recognition %
----------  ---  ---  ---  ---  ---  ---  -------  ---------------
         0   49    9    6    2    0    0       66            74.24
         1   26   14    6    1    1    0       48            29.17
         2   14   16    4    1    1    1       37            10.81
         3   11    6   10   23    6    1       57            40.35
         4    0    1    2   18   20    4       45            44.44
         5    1    0    5    4   11   26       47            55.32


### Naive Bayes Evaluation & Confusion Matrix

In [47]:
import numpy as np
import pandas as pd

X = np.array(X) 
y = np.array(y) 

# Initialize StratifiedKFold with k=10
kf = myevaluation.stratified_kfold_split(X, y, n_splits=10, random_state=None, shuffle=False)

# Initialize lists to store metrics
nb_accuracies, nb_precisions, nb_recalls, nb_f1s, nb_conf_matrices = [], [], [], [], []

# Initialize lists to collect all true and predicted labels across all folds
all_y_true = []
all_y_pred = []

# Define the correct labels for the confusion matrix and metrics
labels = [0, 1, 2, 3, 4, 5]

# Loop over each fold in Stratified K-Fold
for train_index, test_index in kf:
    nb_X_train, nb_X_test = X[train_index], X[test_index]
    nb_y_train, nb_y_test = y[train_index], y[test_index]

    # Initialize and fit MyNaiveBayesClassifier
    nb_model = MyNaiveBayesClassifier()
    nb_model.fit(nb_X_train.tolist(), nb_y_train.tolist())
    
    # Make predictions on the test set
    nb_y_pred = nb_model.predict(nb_X_test.tolist())
    
    # Calculate and store metrics for this fold
    accuracy = myevaluation.accuracy_score(nb_y_test, nb_y_pred)
    error = 1 - accuracy
    precision_score = myevaluation.binary_precision_score(nb_y_test, nb_y_pred, labels=labels)
    recall_score = myevaluation.binary_recall_score(nb_y_test, nb_y_pred, labels=labels)
    f1 = myevaluation.binary_f1_score(nb_y_test, nb_y_pred, labels=labels)
    confusion_matrix = myevaluation.confusion_matrix(nb_y_test, nb_y_pred, labels=labels)

    # Store metrics for this fold
    nb_accuracies.append(accuracy)
    nb_precisions.append(precision_score)
    nb_recalls.append(recall_score)
    nb_f1s.append(f1)
    nb_conf_matrices.append(confusion_matrix)

    # Collect all true and predicted labels for final confusion matrix
    all_y_true.extend(nb_y_test)
    all_y_pred.extend(nb_y_pred)
   

# Calculate the final confusion matrix using all folds' predictions
final_confusion_matrix = myevaluation.confusion_matrix(all_y_true, all_y_pred, labels=labels)
matrix_with_totals = myutils.calculate_confusion_matrix_totals(final_confusion_matrix)

In [48]:
print("===========================================")
print("Naive Bayes Classifier Performance")
print("===========================================")

# Calculate and print average performance
print(f"Accuracy: {np.mean(nb_accuracies):.2f}")
print(f"Error: {1 - np.mean(nb_accuracies):.2f}")
print(f"Precision: {np.mean(nb_precisions):.2f}")
print(f"Recall: {np.mean(nb_recalls):.2f}")
print(f"F1 Score: {np.mean(nb_f1s):.2f}\n")

# Display the confusion matrix
print("Confusion Matrix:")
myutils.display_confusion_matrix(matrix_with_totals, labels)

Naive Bayes Classifier Performance
Accuracy: 0.37
Error: 0.63
Precision: 0.50
Recall: 0.59
F1 Score: 0.53

Confusion Matrix:
  Survived    0    1    2    3    4    5    Total    Recognition %
----------  ---  ---  ---  ---  ---  ---  -------  ---------------
         0   39   10    4    5    4    4       66            59.09
         1   12   15    9    9    2    1       48            31.25
         2    4   10   14    3    2    4       37            37.84
         3    6   10    1   21   10    9       57            36.84
         4   10    2    5    9   11    8       45            24.44
         5   11    1    5   11    8   11       47            23.40


### Random Forest Performance Evaluation & Confusion Matrix

In [49]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold

X = np.array(X)
y = np.array(y)

# Initialize StratifiedKFold with k=10
kf = StratifiedKFold(n_splits=10, shuffle=False, random_state=None)

# Initialize lists to store metrics
rf_accuracies, rf_precisions, rf_recalls, rf_f1s, rf_conf_matrices = [], [], [], [], []

# Initialize lists to collect all true and predicted labels across all folds
all_y_true = []
all_y_pred = []

# Define the correct labels for the confusion matrix and metrics
labels = [0, 1, 2, 3, 4, 5]

# Loop over each fold
for train_index, test_index in kf.split(X, y):
    rf_X_train, rf_X_test = X[train_index], X[test_index]
    rf_y_train, rf_y_test = y[train_index], y[test_index]

    # Initialize and fit RandomForestClassifier
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(rf_X_train, rf_y_train)

    # Make predictions on the test set
    rf_y_pred = rf_model.predict(rf_X_test)

    # Calculate and store metrics for this fold
    accuracy = accuracy_score(rf_y_test, rf_y_pred)
    precision = precision_score(rf_y_test, rf_y_pred, labels=labels, average='macro', zero_division=0)
    recall = recall_score(rf_y_test, rf_y_pred, labels=labels, average='macro', zero_division=0)
    f1 = f1_score(rf_y_test, rf_y_pred, labels=labels, average='macro', zero_division=0)
    conf_matrix = confusion_matrix(rf_y_test, rf_y_pred, labels=labels)

    # Store metrics for this fold
    rf_accuracies.append(accuracy)
    rf_precisions.append(precision)
    rf_recalls.append(recall)
    rf_f1s.append(f1)
    rf_conf_matrices.append(conf_matrix)

    # Collect all true and predicted labels for final confusion matrix
    all_y_true.extend(rf_y_test)
    all_y_pred.extend(rf_y_pred)

# Calculate the final confusion matrix using all folds' predictions
final_confusion_matrix = confusion_matrix(all_y_true, all_y_pred, labels=labels)

# Optionally add functionality to calculate totals for the confusion matrix if required
# matrix_with_totals = myutils.calculate_confusion_matrix_totals(final_confusion_matrix)

# Print or return the results as needed
print("Accuracies:", rf_accuracies)
print("Precisions:", rf_precisions)
print("Recalls:", rf_recalls)
print("F1 Scores:", rf_f1s)
print("Final Confusion Matrix:\n", final_confusion_matrix)


Accuracies: [0.7, 0.9666666666666667, 1.0, 1.0, 1.0, 1.0, 0.9333333333333333, 1.0, 0.9333333333333333, 0.8333333333333334]
Precisions: [0.7157407407407407, 0.9722222222222222, 1.0, 1.0, 1.0, 1.0, 0.9583333333333334, 1.0, 0.9444444444444445, 0.844047619047619]
Recalls: [0.6666666666666666, 0.9583333333333334, 1.0, 1.0, 1.0, 1.0, 0.9166666666666666, 1.0, 0.9249999999999999, 0.8250000000000001]
F1 Scores: [0.6726010101010101, 0.961038961038961, 1.0, 1.0, 1.0, 1.0, 0.9285714285714285, 1.0, 0.9273689273689274, 0.826190476190476]
Final Confusion Matrix:
 [[66  0  0  0  0  0]
 [ 1 47  0  0  0  0]
 [ 0  2 32  3  0  0]
 [ 0  0  2 53  2  0]
 [ 0  0  0  3 39  3]
 [ 1  0  0  0  2 44]]


In [50]:
print("===========================================")
print("Random Forest Classifier Performance")
print("===========================================")

# Calculate and print average performance
print(f"Accuracy: {np.mean(rf_accuracies):.2f}")
print(f"Error: {1 - np.mean(rf_accuracies):.2f}")
print(f"Precision: {np.mean(rf_precisions):.2f}")
print(f"Recall: {np.mean(rf_recalls):.2f}")
print(f"F1 Score: {np.mean(rf_f1s):.2f}\n")

# Display the confusion matrix
print("Confusion Matrix:")
myutils.display_confusion_matrix(matrix_with_totals, labels)

Random Forest Classifier Performance
Accuracy: 0.94
Error: 0.06
Precision: 0.94
Recall: 0.93
F1 Score: 0.93

Confusion Matrix:
  Survived    0    1    2    3    4    5    Total    Recognition %
----------  ---  ---  ---  ---  ---  ---  -------  ---------------
         0   39   10    4    5    4    4       66            59.09
         1   12   15    9    9    2    1       48            31.25
         2    4   10   14    3    2    4       37            37.84
         3    6   10    1   21   10    9       57            36.84
         4   10    2    5    9   11    8       45            24.44
         5   11    1    5   11    8   11       47            23.40


In [51]:
import numpy as np
import pandas as pd

# Load the dataset (assuming `data` is a pandas DataFrame containing the provided sample data)
data = pd.read_csv('normalized_flights.csv') 

# Define delay categories
def categorize_delay(delay):
    if delay <= 0:
        return 0
    elif 0 < delay <= 30:
        return 1
    elif 30 < delay <= 60:
        return 2
    elif 60 < delay <= 120:
        return 3
    elif 120 < delay <= 180:
        return 4
    else:
        return 5

# Map delays to categories
data['delay_category'] = data['arr_delay'].apply(categorize_delay)

# Convert categorical features to integers (label encoding)
categorical_cols = ['carrier', 'flight', 'tailnum', 'origin', 'dest']  # List your categorical columns
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str))  # Convert to string if necessary and apply label encoding
    label_encoders[col] = le  # Store the encoder to use it later if needed

# Prepare features (X) and labels (y)
X = data[['dep_time', 'sched_dep_time', 'dep_delay', 'arr_time', 'sched_arr_time', 'arr_delay', 'air_time', 'distance', 'dest',
          'carrier', 'month', 'hour']].fillna(0)

y = data['delay_category'].values

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

import numpy as np
import pandas as pd
from collections import Counter
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

class DecisionTreeRandomForest:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def predict(self, X):
        return [self._predict_single(self.tree, x) for x in X]

    def _gini(self, y):
        m = len(y)
        if m == 0:
            return 0
        counts = Counter(y)
        return 1 - sum((count / m) ** 2 for count in counts.values())

    def _split(self, X, y, feature, threshold):
        left_indices = [i for i, x in enumerate(X[:, feature]) if x < threshold]
        right_indices = [i for i, x in enumerate(X[:, feature]) if x >= threshold]
        return left_indices, right_indices

    def _information_gain(self, y, left_indices, right_indices):
        m = len(y)
        left_y, right_y = y[left_indices], y[right_indices]
        return self._gini(y) - (len(left_y) / m * self._gini(left_y) + len(right_y) / m * self._gini(right_y))

    def _best_split(self, X, y):
        best_feature, best_threshold, best_gain = None, None, 0
        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_indices, right_indices = self._split(X, y, feature, threshold)
                gain = self._information_gain(y, left_indices, right_indices)
                if gain > best_gain:
                    best_feature, best_threshold, best_gain = feature, threshold, gain
        return best_feature, best_threshold

    def _build_tree(self, X, y, depth=0):
        if len(set(y)) == 1 or depth == self.max_depth:
            return Counter(y).most_common(1)[0][0]

        feature, threshold = self._best_split(X, y)
        if feature is None:
            return Counter(y).most_common(1)[0][0]

        left_indices, right_indices = self._split(X, y, feature, threshold)
        left_subtree = self._build_tree(X[left_indices], y[left_indices], depth + 1)
        right_subtree = self._build_tree(X[right_indices], y[right_indices], depth + 1)
        return (feature, threshold, left_subtree, right_subtree)

    def _predict_single(self, node, x):
        if not isinstance(node, tuple):
            return node
        feature, threshold, left_subtree, right_subtree = node
        if x[feature] < threshold:
            return self._predict_single(left_subtree, x)
        else:
            return self._predict_single(right_subtree, x)

class RandomForest:
    def __init__(self, n_estimators=10, max_depth=None, max_features=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.max_features = max_features
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        n_samples = X.shape[0]
        for _ in range(self.n_estimators):
            bootstrap_indices = np.random.choice(n_samples, n_samples, replace=True)
            bootstrap_X, bootstrap_y = X[bootstrap_indices], y[bootstrap_indices]

            tree = DecisionTreeRandomForest(max_depth=self.max_depth)
            tree.fit(bootstrap_X, bootstrap_y)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        return [Counter(tree_preds[:, i]).most_common(1)[0][0] for i in range(X.shape[0])]

# Main Code

# Convert features and labels to numpy arrays
X = np.array(X)
y = np.array(y)

# Perform manual stratified K-fold
n_splits = 10
indices = np.arange(len(y))
unique_classes, y_counts = np.unique(y, return_counts=True)
folds = {i: [] for i in range(n_splits)}

for cls in unique_classes:
    cls_indices = indices[y == cls]
    np.random.shuffle(cls_indices)
    for i, index in enumerate(cls_indices):
        folds[i % n_splits].append(index)

fold_indices = [np.array(folds[i]) for i in range(n_splits)]

# Metrics storage
rf_accuracies, rf_precisions, rf_recalls, rf_f1s = [], [], [], []

# Iterate through folds
for i in range(n_splits):
    test_indices = fold_indices[i]
    train_indices = np.concatenate([fold_indices[j] for j in range(n_splits) if j != i])

    rf_X_train, rf_X_test = X[train_indices], X[test_indices]
    rf_y_train, rf_y_test = y[train_indices], y[test_indices]

    # Train Random Forest
    rf_model = RandomForest(n_estimators=10, max_depth=5)
    rf_model.fit(rf_X_train, rf_y_train)

    # Predictions
    rf_y_pred = rf_model.predict(rf_X_test)

    # Compute metrics
    accuracy = np.sum(rf_y_test == rf_y_pred) / len(rf_y_test)
    precision = precision_score(rf_y_test, rf_y_pred, average='macro', zero_division=0)
    recall = recall_score(rf_y_test, rf_y_pred, average='macro', zero_division=0)
    f1 = f1_score(rf_y_test, rf_y_pred, average='macro', zero_division=0)

    # Store results
    rf_accuracies.append(accuracy)
    rf_precisions.append(precision)
    rf_recalls.append(recall)
    rf_f1s.append(f1)

    # Debugging outputs
    print(f"Fold {i + 1}")
    print("True Labels:", rf_y_test)
    print("Predicted Labels:", rf_y_pred)

# Print final metrics
print("Accuracies:", rf_accuracies)
print("Precisions:", rf_precisions)
print("Recalls:", rf_recalls)
print("F1 Scores:", rf_f1s)




Fold 1
True Labels: [0 0 0 0 0 0 0 1 1 1 1 1 2 2 2 2 3 3 3 3 3 3 4 4 4 4 4 5 5 5 5 5]
Predicted Labels: [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5]
Fold 2
True Labels: [0 0 0 0 0 0 0 1 1 1 1 1 2 2 2 2 3 3 3 3 3 3 4 4 4 4 4 5 5 5 5 5]
Predicted Labels: [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 1, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 0, 5, 5, 5, 5]
Fold 3
True Labels: [0 0 0 0 0 0 0 1 1 1 1 1 2 2 2 2 3 3 3 3 3 3 4 4 4 4 4 5 5 5 5 5]
Predicted Labels: [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 3, 5, 5, 5, 5, 5]
Fold 4
True Labels: [0 0 0 0 0 0 0 1 1 1 1 1 2 2 2 2 3 3 3 3 3 3 4 4 4 4 4 5 5 5 5 5]
Predicted Labels: [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 4, 5]
Fold 5
True Labels: [0 0 0 0 0 0 0 1 1 1 1 1 2 2 2 2 3 3 3 3 3 3 4 4 4 4 4 5 5 5 5 5]
Predicted Labels: [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 2, 4, 4, 4, 4, 4, 5, 5, 5, 5

In [52]:
print("===========================================")
print("Random Forest Classifier Performance")
print("===========================================")

# Calculate and print average performance
print(f"Accuracy: {np.mean(rf_accuracies):.2f}")
print(f"Error: {1 - np.mean(rf_accuracies):.2f}")
print(f"Precision: {np.mean(rf_precisions):.2f}")
print(f"Recall: {np.mean(rf_recalls):.2f}")
print(f"F1 Score: {np.mean(rf_f1s):.2f}\n")

# Display the confusion matrix
print("Confusion Matrix:")
myutils.display_confusion_matrix(matrix_with_totals, labels)

Random Forest Classifier Performance
Accuracy: 0.98
Error: 0.02
Precision: 0.98
Recall: 0.97
F1 Score: 0.98

Confusion Matrix:
  Survived    0    1    2    3    4    5    Total    Recognition %
----------  ---  ---  ---  ---  ---  ---  -------  ---------------
         0   39   10    4    5    4    4       66            59.09
         1   12   15    9    9    2    1       48            31.25
         2    4   10   14    3    2    4       37            37.84
         3    6   10    1   21   10    9       57            36.84
         4   10    2    5    9   11    8       45            24.44
         5   11    1    5   11    8   11       47            23.40


In [ ]:
class MyDecisionTreeRandomForest:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def predict(self, X):
        return [self._predict_single(self.tree, x) for x in X]

    def _gini(self, y):
        m = len(y)
        if m == 0:
            return 0
        counts = Counter(y)
        return 1 - sum((count / m) ** 2 for count in counts.values())

    def _split(self, X, y, feature, threshold):
        left_indices = [i for i, x in enumerate(X[:, feature]) if x < threshold]
        right_indices = [i for i, x in enumerate(X[:, feature]) if x >= threshold]
        return left_indices, right_indices

    def _information_gain(self, y, left_indices, right_indices):
        m = len(y)
        left_y, right_y = y[left_indices], y[right_indices]
        return self._gini(y) - (len(left_y) / m * self._gini(left_y) + len(right_y) / m * self._gini(right_y))

    def _best_split(self, X, y):
        best_feature, best_threshold, best_gain = None, None, 0
        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_indices, right_indices = self._split(X, y, feature, threshold)
                gain = self._information_gain(y, left_indices, right_indices)
                if gain > best_gain:
                    best_feature, best_threshold, best_gain = feature, threshold, gain
        return best_feature, best_threshold

    def _build_tree(self, X, y, depth=0):
        if len(set(y)) == 1 or depth == self.max_depth:
            return Counter(y).most_common(1)[0][0]

        feature, threshold = self._best_split(X, y)
        if feature is None:
            return Counter(y).most_common(1)[0][0]

        left_indices, right_indices = self._split(X, y, feature, threshold)
        left_subtree = self._build_tree(X[left_indices], y[left_indices], depth + 1)
        right_subtree = self._build_tree(X[right_indices], y[right_indices], depth + 1)
        return (feature, threshold, left_subtree, right_subtree)

    def _predict_single(self, node, x):
        if not isinstance(node, tuple):
            return node
        feature, threshold, left_subtree, right_subtree = node
        if x[feature] < threshold:
            return self._predict_single(left_subtree, x)
        else:
            return self._predict_single(right_subtree, x)

class MyRandomForest:
    def __init__(self, n_estimators=10, max_depth=None, max_features=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.max_features = max_features
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        n_samples = X.shape[0]
        for _ in range(self.n_estimators):
            bootstrap_indices = np.random.choice(n_samples, n_samples, replace=True)
            bootstrap_X, bootstrap_y = X[bootstrap_indices], y[bootstrap_indices]

            tree = DecisionTreeRandomForest(max_depth=self.max_depth)
            tree.fit(bootstrap_X, bootstrap_y)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        return [Counter(tree_preds[:, i]).most_common(1)[0][0] for i in range(X.shape[0])]

# Main Code

# Convert features and labels to numpy arrays
X = np.array(X)
y = np.array(y)

# Perform manual stratified K-fold
n_splits = 10
indices = np.arange(len(y))
unique_classes, y_counts = np.unique(y, return_counts=True)
folds = {i: [] for i in range(n_splits)}

for cls in unique_classes:
    cls_indices = indices[y == cls]
    np.random.shuffle(cls_indices)
    for i, index in enumerate(cls_indices):
        folds[i % n_splits].append(index)

fold_indices = [np.array(folds[i]) for i in range(n_splits)]

# Metrics storage
rf_accuracies, rf_precisions, rf_recalls, rf_f1s = [], [], [], []

# Iterate through folds
for i in range(n_splits):
    test_indices = fold_indices[i]
    train_indices = np.concatenate([fold_indices[j] for j in range(n_splits) if j != i])

    rf_X_train, rf_X_test = X[train_indices], X[test_indices]
    rf_y_train, rf_y_test = y[train_indices], y[test_indices]

    # Train Random Forest
    rf_model = MyRandomForest(n_estimators=10, max_depth=5)
    rf_model.fit(rf_X_train, rf_y_train)

    # Predictions
    rf_y_pred = rf_model.predict(rf_X_test)

    # Compute metrics
    accuracy = np.sum(rf_y_test == rf_y_pred) / len(rf_y_test)
    precision = myevaluation.binary_precision_score(rf_y_test, rf_y_pred)
    recall = myevaluation.binary_recall_score(rf_y_test, rf_y_pred)
    f1 = myevaluation.binary_f1_score(rf_y_test, rf_y_pred)

    # Store results
    rf_accuracies.append(accuracy)
    rf_precisions.append(precision)
    rf_recalls.append(recall)
    rf_f1s.append(f1)

    # Debugging outputs
    print(f"Fold {i + 1}")
    print("True Labels:", rf_y_test)
    print("Predicted Labels:", rf_y_pred)

# Print final metrics
print("Accuracies:", rf_accuracies)
print("Precisions:", rf_precisions)
print("Recalls:", rf_recalls)
print("F1 Scores:", rf_f1s)


Fold 1
True Labels: [0 0 0 0 0 0 0 1 1 1 1 1 2 2 2 2 3 3 3 3 3 3 4 4 4 4 4 5 5 5 5 5]
Predicted Labels: [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5]
Fold 2
True Labels: [0 0 0 0 0 0 0 1 1 1 1 1 2 2 2 2 3 3 3 3 3 3 4 4 4 4 4 5 5 5 5 5]
Predicted Labels: [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5]
Fold 3
True Labels: [0 0 0 0 0 0 0 1 1 1 1 1 2 2 2 2 3 3 3 3 3 3 4 4 4 4 4 5 5 5 5 5]
Predicted Labels: [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5]
Fold 4
True Labels: [0 0 0 0 0 0 0 1 1 1 1 1 2 2 2 2 3 3 3 3 3 3 4 4 4 4 4 5 5 5 5 5]
Predicted Labels: [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5]
Fold 5
True Labels: [0 0 0 0 0 0 0 1 1 1 1 1 2 2 2 2 3 3 3 3 3 3 4 4 4 4 4 5 5 5 5 5]
Predicted Labels: [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5

In [54]:
print("===========================================")
print("Random Forest Classifier Performance")
print("===========================================")

# Calculate and print average performance
print(f"Accuracy: {np.mean(rf_accuracies):.2f}")
print(f"Error: {1 - np.mean(rf_accuracies):.2f}")
print(f"Precision: {np.mean(rf_precisions):.2f}")
print(f"Recall: {np.mean(rf_recalls):.2f}")
print(f"F1 Score: {np.mean(rf_f1s):.2f}\n")

# Display the confusion matrix
print("Confusion Matrix:")
myutils.display_confusion_matrix(matrix_with_totals, labels)

Random Forest Classifier Performance
Accuracy: 0.98
Error: 0.02
Precision: 0.96
Recall: 1.00
F1 Score: 0.98

Confusion Matrix:
  Survived    0    1    2    3    4    5    Total    Recognition %
----------  ---  ---  ---  ---  ---  ---  -------  ---------------
         0   39   10    4    5    4    4       66            59.09
         1   12   15    9    9    2    1       48            31.25
         2    4   10   14    3    2    4       37            37.84
         3    6   10    1   21   10    9       57            36.84
         4   10    2    5    9   11    8       45            24.44
         5   11    1    5   11    8   11       47            23.40
